In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [3]:
df = pd.read_csv('outputs/dataset_20210523.csv')
df_champions = pd.read_csv('data/dataset_champions.csv')
df

,game,blueTop,blueTopGP,blueTopWR,blueTopKDA,blueJungle,blueJungleGP,blueJungleWR,blueJungleKDA,blueMid,...,redMidKDA,redAdc,redAdcGP,redAdcWR,redAdcKDA,redSupport,redSupportGP,redSupportWR,redSupportKDA,result
0,ESPORTSTMNT03/1632489,82,8,0.50,2.9,104,27,0.85,5.9,142,...,3.6,81,21,0.57,4.3,89,10,0.60,2.5,0
1,ESPORTSTMNT03/1632500,79,14,0.36,2.6,104,63,0.63,5.1,68,...,4.9,21,10,0.70,5.3,3,11,0.55,2.8,1
2,ESPORTSTMNT03/1632502,79,14,0.36,2.6,104,63,0.63,5.1,142,...,4.4,21,10,0.70,5.3,3,11,0.55,2.8,1
3,6909-9183,43,2,0.00,0.9,80,5,0.60,3.9,134,...,5.2,145,43,0.60,6.3,3,4,0.75,4.5,1
4,6909-9184,24,16,0.56,2.8,120,11,0.55,3.6,61,...,5.2,145,43,0.60,6.3,57,2,0.50,1.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3674,ESPORTSTMNT02/1720376,122,5,0.40,3.0,77,11,0.73,4.1,112,...,4.1,236,3,1.00,18.5,12,17,0.59,3.9,1
3675,ESPORTSTMNT02/1720377,79,3,0.00,0.6,421,1,0.00,1.4,134,...,0.0,498,15,0.67,6.5,412,14,0.57,3.2,1
3676,ESPORTSTMNT02/1720384,150,10,0.50,2.8,77,9,0.56,2.4,112,...,2.4,523,6,0.33,4.1,79,2,0.50,3.2,0
3677,ESPORTSTMNT02/1720392,517,1,0.00,1.3,120,4,0.00,1.9,134,...,2.8,18,11,0.64,4.3,12,24,0.63,2.8,1


In [15]:
y = df['result'].copy()
X = df.drop(['blueTop', 'blueJungle', 'blueMid', 'blueADC', 'blueSupport', 'redTop', 'redJungle', 'redMid', 'redAdc', 'redSupport', 'result', 'game'],axis=1)
X

,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,8,0.50,2.9,27,0.85,5.9,59,0.76,10.7,10,...,4.3,13,0.46,3.6,21,0.57,4.3,10,0.60,2.5
1,14,0.36,2.6,63,0.63,5.1,1,0.00,1.0,20,...,4.1,55,0.69,4.9,10,0.70,5.3,11,0.55,2.8
2,14,0.36,2.6,63,0.63,5.1,5,0.40,2.9,2,...,6.4,3,0.67,4.4,10,0.70,5.3,11,0.55,2.8
3,2,0.00,0.9,5,0.60,3.9,36,0.75,6.2,5,...,3.0,6,0.67,5.2,43,0.60,6.3,4,0.75,4.5
4,16,0.56,2.8,11,0.55,3.6,50,0.64,6.1,5,...,2.9,6,0.67,5.2,43,0.60,6.3,2,0.50,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3674,5,0.40,3.0,11,0.73,4.1,0,0.00,0.0,8,...,10.3,12,0.50,4.1,3,1.00,18.5,17,0.59,3.9
3675,3,0.00,0.6,1,0.00,1.4,12,0.50,4.1,26,...,4.1,0,0.00,0.0,15,0.67,6.5,14,0.57,3.2
3676,10,0.50,2.8,9,0.56,2.4,15,0.67,3.6,21,...,3.3,12,0.42,2.4,6,0.33,4.1,2,0.50,3.2
3677,1,0.00,1.3,4,0.00,1.9,3,0.00,1.5,7,...,4.9,38,0.32,2.8,11,0.64,4.3,24,0.63,2.8


In [16]:
def preprocess_input(X,y):
    skf = StratifiedKFold(n_splits=10)
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (Linear Kernel) trained
C:\Users\linco\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [18]:
scores_list = []
for name,model in models.items():
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [19]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.922554,0.920118,0.912023,0.916053
1,Support Vector Machine (Linear Kernel),0.923913,0.920354,0.914956,0.917647
2,Support Vector Machine (RBF Kernel),0.927989,0.921053,0.923754,0.922401
3,Decission Tree,0.851902,0.853659,0.821114,0.837070
4,Adaboost,0.932065,0.924198,0.929619,0.926901
5,Random Forest,0.938859,0.948485,0.917889,0.932936
6,Gradient Boosting Classifier,0.937500,0.945619,0.917889,0.931548
